----
<img src="../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Session layer
Before you can retreive any data with the Refinitiv Data Library, your must open a data session to manage your connection to the Refinitv Data Platform. Depending on the way you intend to access the platform you must open either a Platform Session (for direct connection) or a Desktop Session (for a connection via Workspace or via CodeBook). This notebook demonstrates different options to create and open Refinitiv Data Library sessions.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data import session
import datetime

## User details and other information
Please set the following variables with your user information and environment details 

In [3]:
APP_KEY = 'YOUR APP KEY GOES HERE!'
RDP_LOGIN                   = 'YOUR RDP LOGIN OR MACHINE GOES HERE!'
RDP_PASSWORD                = 'YOUR RDP PASSWORD GOES HERE!'
REALTIME_DISTRIBUTION_SERVICE_HOST      = 'ADS IP'
REALTIME_DISTRIBUTION_SERVICE_USER_NAME = 'DACS USER NAME'

## The different types of data sessions

Depending on the access point your application uses to connect to the Refinitiv Data Platform, it has to create and open either a Platform session or a Desktop session: 

 - **Platform session:** This type of session is used to connect to the Refinitiv Data Platform either directly or via a local Refinitiv's Real-Time Distribution System. The Platform session requires Refinitiv Data Platform credentials (either a user account or a machine account) or connection details for your local Real-Time Distribution System (IP of the system and DACS user name) or both.
    - When the Platform session is used with Refinitiv Data Platform credentials only, it retrieves both streaming data and non-streaming data from the Refinitiv Data Platform.
    - When the Platform session is used with connection details of your Real-Time Distribution System only, it is able to retrieve streaming data only.
    - When the Platform session is used with both Refinitiv Data Platform credentials and connection details of your Real-Time Distribution System, it retrieves streaming data from the Real-Time Distribution System and non-streaming data from the Refinitiv Data Platform.
 - **Desktop session:** This type of session is used to connect the Refinitiv Data Platform either via Eikon, via Refinitiv Workspace or via CodeBook. It requires either Eikon Desktop or Workspace Desktop to be running alongside your Python script.
 
These 2 types of session also require an App Key that uniquely identifies your application. Please refer to the Refinitiv Data Libraries Quick Start guide to learn how to create an App Key for your application.

## Managing data sessions using the "Access" layer

The Access of the library provides you with two functions - open_session() and close_session() - that greatly simplify data session management. These functions are based on the Session layer described later in this notebook and they rely on the information contained in the configuration of the library. Depending on your use case, you can either rely on the default configuration of the library (Workspace Desktop use case or CodeBook use case), or you can define your own configuration file to define any specific parameters for you session (Refinitiv Data Platform use case or Real-Time Distribution System use case).

### Opening a session 

The open_session() function creates and open a session based on the information contained in the configuration of the library. By default, it creates a Desktop Session that works out of the box with Refinitiv Workspace and CodeBook. If you need a Platform Session instead or if you need to set specific parameters for your session, you must create a refinitiv-data.config.json to define the session type you need and its parameters. The open_session() function will read this configuration file and create the session of your choice. 

_**Note:** With Eikon and Workspace versions prior to February 2022, the refinitiv-data.config.json configuration file is required for both Platform sessions and Desktop sessions. When used with a Desktop session, this file must contain your App Key._

Once the session created, open_session() registers it as the default session of the library. Meaning that this session will be used by all the other part of the library unless you explicitely specify different session. When open_session() is called several times, the latest created session becomes the default session.

#### Open the default session

This call opens the default session defined in the configuration of the library

In [4]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f88ec116250 {name='workspace'}>

#### Open the default with a specific App Key  

This call opens a session indentified by its name. This session must be defined in the configuration of the library

In [5]:
rd.open_session(app_key = APP_KEY)

<refinitiv.data.session.Definition object at 0x7f88eacc28d0 {name='workspace'}>

#### Open a session by name 

This call opens a session indentified by its name. This session must be defined in the configuration of the library

In [6]:
rd.open_session('platform.rdp')

<refinitiv.data.session.platform.Definition object at 0x7f88ec10d750 {session_name='rdp'}>

#### Open a session defined in a custom configuration file 

This call opens the default session as defined in the configuration file passed as parameter.

In [7]:
rd.open_session(config_name='../../Configuration/refinitiv-data.custom.config.json')

<refinitiv.data.session.platform.Definition object at 0x7fd235bbea90 {session_name='my-custom-session'}>

#### Close the default session 

This call closes the default session of the library

In [8]:
rd.close_session()

#### Close a specific session 

In [9]:
# Create 2 sessions
desktop_session = rd.open_session('desktop.workspace')
platform_session = rd.open_session('platform.rdp') # platform_session becomes the default session because it is the last created session 

# Close the default session that is platform_session
rd.close_session()

# Close desktop_session. This must be done explicitely because desktop_session is not the default session
desktop_session.close()

<OpenState.Closed: 1>

## Managing data sessions using the "Session" layer

The "Session" layer offers you more control for session management and does not necessarily require a configuration file. 

### Managing sessions based on a config file

#### Create the default session

This call creates the default session defined in the configuration of the library

In [10]:
session = rd.session.Definition().get_session()

#### Create a desktop session by name

This call creates the desktop session named "workspace" as defined in the configuration of the library

In [11]:
session = rd.session.desktop.Definition("workspace").get_session()

#### Create a platform session by name

This call creates the Platform session named "rdp" as defined in the configuration of the library

In [12]:
session = rd.session.platform.Definition("rdp").get_session()

#### Create a session by full name

In [13]:
session = rd.session.Definition('desktop.workspace').get_session()

#### Set the default session

In [14]:
rd.session.set_default(session)

#### Get the default session

In [15]:
session = rd.session.get_default()

### Managing sessions without any config file

#### Create a Desktop session using with an App Key
For a connection via Refinitiv Workspace, Eikon or CodeBook 

In [16]:
session = rd.session.desktop.Definition(app_key = APP_KEY).get_session()

#### Create a Platform session using with an App Key and GrantPassword
For a direct connection to the Refinitiv Data Platform

In [17]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
).get_session()

#### Create a Platform session using with an App Key and deployed_platform parameters 
For a direct connection to a Refinitiv Realtime Distribution System (a.k.a.TREP)

In [18]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    deployed_platform_host = REALTIME_DISTRIBUTION_SERVICE_HOST,
    deployed_platform_username = REALTIME_DISTRIBUTION_SERVICE_USER_NAME
).get_session()

#### Create a Platform session using with an App Key, a GrantPassword and deployed_platform parameters 
For a connection to the Refinitiv Data Platform and a connection to a Refinitiv Realtime Distribution System. With this configuration, streaming data is retrieved from the Refinitiv Realtime Distribution System while non-streaming data is retrieved from the Refinitiv Data Platform.

In [19]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    ),
    deployed_platform_host = REALTIME_DISTRIBUTION_SERVICE_HOST,
    deployed_platform_username = REALTIME_DISTRIBUTION_SERVICE_USER_NAME
).get_session()

## Open the session

In [18]:
session.open()

<OpenState.Open: 3>

## Check the open state

In [19]:
session.open_state

<OpenState.Open: 3>

## Close the session

In [20]:
session.close()

<OpenState.Closed: 1>